In [26]:
from tqdm.auto import tqdm
import math
from decimal import Decimal
import time
import numpy as np

In [27]:
# Define the PRIME_SET and the Prime function
PRIME_SET = set([2])
# Define the FIB_LIST
FIB_LIST = [1,1]

In [28]:
RHO_MAP: dict[int, list[tuple[int, int]]] = {}

In [29]:
def break_down(n: int, base: int) -> tuple[int, list[tuple[int, int]]]:
  exponent = 0
  while n % base == 0:
    n = n // base
    exponent += 1
  if exponent > 0:
    return n, [(base, exponent)]
  return n, []

In [30]:
def is_prime(k: int, add_to_PRIME_SET: bool = False):
  for prime in PRIME_SET:
    if k % prime == 0 and prime != k:
      return False
  if add_to_PRIME_SET:
    PRIME_SET.add(k)
  return True

In [31]:
# Define the Rho function
def rho(n: int, enable_tqdm: bool = False, trace: bool = False):
  pairs: list[tuple[int, int]] = []
  number = n
  range_bar = tqdm(
      total=n,
      initial=2,
      disable=not enable_tqdm,
      # bar_format="{l_bar}{bar} {n_fmt}/{total_fmt}",
      unit_scale=True)
  inc = 1
  base = 2
  while base < n:
    if base in PRIME_SET or is_prime(base, add_to_PRIME_SET=True):
      if number == 1:
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      if base > Decimal(n).sqrt():
        pairs.append((number, 1))
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      if number < n:
        pairs = [*pairs, *rho(number, enable_tqdm and trace, trace)]
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      number, pair = break_down(number, base)
      pairs.extend(pair)
    base += inc
    range_bar.update(inc)
  if number > 1:
    pairs.append((number, 1))
  range_bar.close()
  return pairs

In [32]:
# Define the Rho function
def rho_base(n: int, base: int = 2, inc: int = 1, dynamic: bool = True, enable_tqdm: bool = False, trace: bool = False) -> list[tuple[int, int]]:
  pairs: list[tuple[int, int]] = RHO_MAP.get(n, []) if dynamic else []
  number = n
  range_bar = tqdm(
      total=n,
      initial=base,
      disable=not enable_tqdm,
      unit_scale=True,
  )
  root = np.sqrt(n)

  if pairs:
      return pairs

  while base < n:
    if base in PRIME_SET or is_prime(base, add_to_PRIME_SET=True):
      if number == 1:
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      if base > root:
        pairs.append((number, 1))
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      if number < n:
        rho_bases = rho_base(number,
                              base,
                              inc,
                              dynamic,
                              enable_tqdm and trace,
                              trace)
        pairs.extend(rho_bases)
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      number, pair = break_down(number, base)
      pairs.extend(pair)
    base += inc
    range_bar.update(inc)
  if number > 1:
    pairs.append((number, 1))
  range_bar.close()
  return pairs

In [33]:
# Define the Rho function
def rho_prob_base(n: int, base: int = 2, inc: int = 1, threshold: int = 10**6, enable_tqdm: bool = False, trace: bool = False) -> \
                  tuple[list[tuple[int, int]], float]:
  pairs: list[tuple[int, int]] = RHO_MAP.get(n, [])
  number = n
  range_bar = tqdm(
      total=n,
      initial=base,
      disable=not enable_tqdm,
      unit_scale=True,
  )
  root = np.sqrt(n)

  def calc_prob():
    return min(1.00, (threshold / pairs[-1][0])) if pairs else 1.00

  if pairs:
      return pairs, calc_prob()

  while base < n:
    if base in PRIME_SET or is_prime(base, add_to_PRIME_SET=True):
      if number == 1:
        range_bar.update(n - base)
        range_bar.close()
        return pairs, calc_prob()
      if base > root or base > threshold:
        pairs.append((number, 1))
        range_bar.update(n - base)
        range_bar.close()
        return pairs, calc_prob()
      if number < n:
        rho_bases, _ = rho_prob_base(number,
                                        base,
                                        inc,
                                        threshold,
                                        enable_tqdm and trace,
                                        trace)
        pairs.extend(rho_bases)
        range_bar.update(n - base)
        range_bar.close()
        return pairs, calc_prob()
      number, pair = break_down(number, base)
      pairs.extend(pair)
    base += inc
    range_bar.update(inc)
  if number > 1:
    pairs.append((number, 1))
  range_bar.close()
  return pairs, calc_prob()

In [34]:
RHO_MAP_SET: dict[int, set[tuple[int, int]]] = {}

In [35]:
# Define the Rho function using dynamic programming techniques and with a set definition
def rho_base_dynamic_set(n: int, base: int = 2, inc: int = 1, enable_tqdm: bool = False, trace: bool = False) -> set[tuple[int, int]]:
  pairs: set[tuple[int, int]] = RHO_MAP_SET.get(n, set())
  number = n
  range_bar = tqdm(
      total=n,
      initial=base,
      disable=not enable_tqdm,
      unit_scale=True,
  )
  root = np.sqrt(n)

  if pairs:
      return pairs

  while base < n:
    if base in PRIME_SET or is_prime(base, add_to_PRIME_SET=True):
      if number == 1:
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      if base > root:
        pairs.add((number, 1))
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      if number < n:
        rho_bases = rho_base_dynamic_set(number,
                              base,
                              inc,
                              enable_tqdm and trace,
                              trace)
        pairs = pairs.union(rho_bases)
        range_bar.update(n - base)
        range_bar.close()
        return pairs
      number, pair = break_down(number, base)
      pairs = pairs.union(pair)
    base += inc
    range_bar.update(inc)
  if number > 1:
    pairs.add((number, 1))
  range_bar.close()
  RHO_MAP_SET[n] = pairs
  return pairs

In [36]:
def fib(n: int): # Zero-indexed
  if n < len(FIB_LIST):
    return FIB_LIST[n]
  for i in range(len(FIB_LIST), n+1):
    FIB_LIST.append(FIB_LIST[i-1] + FIB_LIST[i-2])
  return FIB_LIST[n]

In [37]:
def rho_union(*args: list[tuple[int, int]], enable_tqdm: bool = False) -> list[tuple[int,int]]:
  union_list = [*args[0]]
  progress_bar = tqdm(desc="list", total=len(args[1:]), disable=not enable_tqdm)
  for rho_right in args[1:]:
    for pair in rho_right:
      base, exp = pair
      indices = [i for i,x in enumerate(union_list) if base == x[0]]
      index = indices[0] if indices else None
      if index is not None:
        union_list[index] = (union_list[index][0], union_list[index][1] + exp)
      else:
        union_list.append(pair)
    progress_bar.update()
  progress_bar.close()
  return union_list

In [111]:
import big_o
best, others = big_o.big_o(lambda n: rho_base(n, dynamic=False), big_o.datagen.n_, min_n=10**6, max_n=10**10, n_measures=5000, classes=[big_o.complexities.Logarithmic, big_o.complexities.Polynomial, big_o.complexities.Exponential])
print(big_o.reports.big_o_report(best, others))

Best : Logarithmic: time = -0.038 + 0.002*log(n) (sec)              
Logarithmic: time = -0.038 + 0.002*log(n) (sec)                 (res: 0.81)
Polynomial: time = 2.3E-06 * x^0.31 (sec)                       (res: 0.93)
Exponential: time = 0.0013 * 1^n (sec)                          (res: 0.93)

